In [6]:
import pandas as pd
import numpy as np
import riskfolio as rp

# 가상의 자산 수익률 데이터 생성 (12개의 자산, 1000일의 수익률)
np.random.seed(42)
n_assets = 12
n_obs = 1000

assets = ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'BAC', 'WFC', 'XOM', 'CVX', 'TSLA', 'GE', 'CAT', 'MMM']
returns = np.random.randn(n_obs, n_assets) / 100  # 무작위 수익률 데이터

Y = pd.DataFrame(returns, columns=assets)


In [7]:
import pandas as pd

# 자산과 산업 섹터를 정의
asset_classes = {
    'Assets': ['AAPL', 'MSFT', 'GOOGL', 'JPM', 'BAC', 'WFC', 'XOM', 'CVX', 'TSLA', 'GE', 'CAT', 'MMM'],
    'Industry': ['Tech', 'Tech', 'Tech', 'Finance', 'Finance', 'Finance', 
                 'Energy', 'Energy', 'Industrials', 'Industrials', 'Industrials', 'Industrials']
}

asset_classes = pd.DataFrame(asset_classes)


In [8]:
views = {
    'Disabled': [False, False],               # 뷰가 활성화 되어 있음
    'Type': ['Classes', 'Classes'],           # 뷰의 타입은 자산 클래스에 기반
    'Set': ['Industry', 'Industry'],          # 산업 섹터를 기준으로 뷰를 설정
    'Position': ['Tech', 'Energy'],           # Tech 섹터와 Energy 섹터에 대한 뷰를 정의
    'Sign': ['>=', '<='],                     # Tech 섹터는 상대적으로 더 높게, Energy 섹터는 더 낮게 예상
    'Weight': [0.02, -0.01],                  # Tech는 2% 이상, Energy는 1% 이하 수익률 차이를 예상
    'Type Relative': ['Classes', 'Classes'],  # 비교하는 상대도 자산 클래스
    'Relative Set': ['Industry', 'Industry'], # 비교 기준도 산업 섹터
    'Relative': ['Finance', 'Industrials']    # Tech는 Finance와, Energy는 Industrials와 비교
}

views = pd.DataFrame(views)


In [9]:
import riskfolio as rp

# 투자자의 뷰를 기반으로 P와 Q 매트릭스 생성
P, Q = rp.assets_views(views, asset_classes)
display(P)
display(Q)

array([[ 0.33333333,  0.33333333,  0.33333333, -0.33333333, -0.33333333,
        -0.33333333,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [-0.        , -0.        , -0.        , -0.        , -0.        ,
        -0.        , -0.5       , -0.5       ,  0.25      ,  0.25      ,
         0.25      ,  0.25      ]])

array([[0.02],
       [0.01]])

In [18]:
import numpy as np
import pandas as pd
import riskfolio as rp

# 가상의 자산 수익률 데이터 생성 (12개의 자산, 1000일의 수익률)
np.random.seed(42)
n_assets = 12
n_obs = 1000

assets = ['Asset ' + str(i) for i in range(1, n_assets + 1)]
returns = np.random.randn(n_obs, n_assets) / 100  # 무작위 수익률 데이터

Y = pd.DataFrame(returns, columns=assets)

# 자산 클래스 정의
asset_classes = {
    'Assets': assets,
    'Industry': ['Tech', 'Tech', 'Health', 'Health', 'Energy', 'Energy', 
                 'Finance', 'Finance', 'Utilities', 'Utilities', 'Industrials', 'Industrials']
}

asset_classes = pd.DataFrame(asset_classes)

# 투자자의 뷰 정의
views = {
    'Disabled': [False, False],
    'Type': ['Classes', 'Classes'],
    'Set': ['Industry', 'Industry'],
    'Position': ['Tech', 'Energy'],
    'Sign': ['>=', '<='],
    'Weight': [0.02, -0.01],  # Tech의 예상 초과수익률 2%, Energy의 예상 초과수익률 -1%
    'Type Relative': ['Classes', 'Classes'],
    'Relative Set': ['Industry', 'Industry'],
    'Relative': ['Finance', 'Industrials']
}

views = pd.DataFrame(views)

# P와 Q 매트릭스 생성
P, Q = rp.assets_views(views, asset_classes)

display(P)
display(Q)

# 포트폴리오 객체 생성
port = rp.Portfolio(returns=Y)

# 자산 기대 수익률과 공분산 행렬 추정
port.assets_stats(method_mu='hist', method_cov='hist')

# 초기 자산 가중치를 설정
initial_weights = np.array([1/n_assets] * n_assets).reshape(-1, 1)  # 균등 가중치로 초기화, 올바른 차원으로 변환

print(initial_weights)
# Black-Litterman 모델에 맞춘 통계값 계산
port.blacklitterman_stats(P, Q, rf=0.01, w=initial_weights, delta=None, eq=True)

port.solvers = ['MOSEK']  # 최적화에 사용할 솔버 설정
# 최적의 포트폴리오 계산
w_bl = port.optimization(model='BL', rm='MV', obj='Sharpe', rf=0, l=0, hist=False)

# 최적의 포트폴리오 가중치 출력
print("최적화된 포트폴리오 가중치:")
display(w_bl.T)


array([[ 0.5,  0.5,  0. ,  0. ,  0. ,  0. , -0.5, -0.5,  0. ,  0. ,  0. ,
         0. ],
       [-0. , -0. , -0. , -0. , -0.5, -0.5, -0. , -0. , -0. , -0. ,  0.5,
         0.5]])

array([[0.02],
       [0.01]])

[[0.08333333]
 [0.08333333]
 [0.08333333]
 [0.08333333]
 [0.08333333]
 [0.08333333]
 [0.08333333]
 [0.08333333]
 [0.08333333]
 [0.08333333]
 [0.08333333]
 [0.08333333]]
최적화된 포트폴리오 가중치:


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11,Asset 12
weights,0.1557,0.30609,0.121569,1.489530e-09,4.068372e-10,6.654328e-10,1.997796e-10,1.823195e-10,0.160399,0.030897,0.005235,0.220109
